TAsk:Make a code generation of data modelling tested against Iris dataset

Code generation for data modeling typically involves generating code that defines the structure of a database, tables, and relationships based on a given data model. In this case, I'll provide an example using Python to generate a SQL script for creating a table that could represent the Iris dataset. Keep in mind that this is a simplified example, and in a real-world scenario, you might want to include additional considerations, data types, and constraints.

In [1]:
! pip install opencv-python
! pip install pandas
! pip instal nltk
! pip install spacy
! python -m spacy download en_core_web_sm

ERROR: unknown command "instal" - maybe you meant "install"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import cv2
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize,wordpunct_tokenize
from google.colab.patches import cv2_imshow
nltk.download('punkt')
import spacy
from spacy import displacy
from spacy.matcher import matcher
from spacy.tokens import span
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Identify data source in Tableau

In [3]:
from google.colab import files
file="tableau_data.csv"
if file not in '/content/':
      files.upload()

Saving tableau_data.csv to tableau_data.csv
Saving formulas.xlsx to formulas.xlsx


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#! cd /content/drive/My\ Drive/Colab_2024
#! cp /content/drive/My\ Drive/Colab_2024/*.png /content/
#! ls -all

# TASK: FORMULA CONVERSION

TASK 1: Leverage NLP for Tableau formulas identification

Leveraging Natural Language Processing (NLP) for Tableau formulas identification involves using language models to interpret and understand natural language descriptions of analytical tasks and then automatically generating corresponding Tableau formulas. While this is a challenging task, it's feasible with the help of advanced NLP models. Below are general steps to approach this task:

1. Define Use Cases:
Clearly define the use cases for which you want to leverage NLP for Tableau formulas identification. For example, you might want to interpret natural language descriptions and generate Tableau formulas for common tasks like aggregations, calculations, or filtering.

2. Data Collection:
Collect a dataset that includes examples of natural language descriptions paired with their corresponding Tableau formulas. The dataset should cover a variety of scenarios to train a model that can generalize well.

In [6]:

df=pd.read_excel("formulas.xlsx")
print(df.head(5))


                                  Description        Formula (Tableau)
0                         Sum of Sepal Length        SUM(Sepal Length)
1      Average Sepal Width across all entries         AVG(Sepal Width)
2                Find the median Petal Length     MEDIAN(Petal Length)
3  Count the number of records in the dataset                 COUNT(*)
4       Count distinct species in the dataset  COUNT DISTINCT(Species)


In [7]:
df.columns

Index(['Description', 'Formula (Tableau)'], dtype='object')

3. Preprocess the Data:
Clean and preprocess the collected dataset. Tokenize the natural language descriptions and normalize the Tableau formulas to create input-output pairs for training the NLP model.

# Tokenization of the formulas.xlsx dataset

In [8]:
dg_tok=df.apply(lambda x:[wordpunct_tokenize(y) for y in x])

In [9]:
dg_tok=dg_tok.rename(columns=lambda x:"Tokenized " + x)
dg_tok=dg_tok.loc[:,dg_tok.columns[[0,1]]]
dg_tok.rename(columns=lambda x:x.strip(), inplace=True)
print(dg_tok.head(5))


                               Tokenized Description  \
0                           [Sum, of, Sepal, Length]   
1      [Average, Sepal, Width, across, all, entries]   
2                 [Find, the, median, Petal, Length]   
3  [Count, the, number, of, records, in, the, dat...   
4       [Count, distinct, species, in, the, dataset]   

        Tokenized Formula (Tableau)  
0        [SUM, (, Sepal, Length, )]  
1         [AVG, (, Sepal, Width, )]  
2     [MEDIAN, (, Petal, Length, )]  
3                      [COUNT, (*)]  
4  [COUNT, DISTINCT, (, Species, )]  


#Extracting symbols from Formulas (Tableau)

In [10]:

def gather_sym(data,column_name):
    dp=data[column_name]
    dp_text=[]
    dp_sym=[]
    for row in dp:
         drow_text=[]
         drow_sym =[]
         for item in row:
              if re.match(r'\w+',item):
                    drow_text.append(item)
              else:
                    drow_sym.append(item)
         dp_text.append(drow_text)
         dp_sym.append(drow_sym)
    return dp_text,dp_sym


In [11]:
dg_formulas_text,dg_formulas_sym=gather_sym(dg_tok,dg_tok.columns[1])

In [12]:
print(dg_formulas_text)

[['SUM', 'Sepal', 'Length'], ['AVG', 'Sepal', 'Width'], ['MEDIAN', 'Petal', 'Length'], ['COUNT'], ['COUNT', 'DISTINCT', 'Species'], ['MIN', 'Sepal', 'Length'], ['MAX', 'Petal', 'Width'], ['GROUP', 'BY', 'Species'], ['ORDER', 'BY', 'Sepal', 'Length', 'ASC'], ['ORDER', 'BY', 'Species', 'ASC'], ['Sepal', 'Length', '5', '0'], ['Petal', 'Length', 'BETWEEN', '1', '5', 'AND', '2', '5'], ['TOP', '3', 'BY', 'Sepal', 'Width', 'DESC'], ['Species', 'Setosa'], ['Species', 'STARTS', 'WITH', 'Se'], ['SUM', 'Petal', 'Length'], ['AVG', 'Petal', 'Width'], ['MEDIAN', 'Sepal', 'Width'], ['COUNT', 'Sepal', 'Length'], ['COUNTD', 'Species'], ['MIN', 'Sepal', 'Length'], ['MAX', 'Petal', 'Width'], ['SORT', 'Species', 'DESC'], ['SORT', 'Sepal', 'Length', 'ASC'], ['Sepal', 'Length', '4', '5'], ['Petal', 'Width', '2', '0'], ['Sepal', 'Width', '3', '0'], ['ENDSWITH', 'Species', 'ica'], ['Sepal', 'Length', '5', '0'], ['INDEX', '5', 'SORT', 'BY', 'SUM', 'Sepal', 'Width', 'DESC'], ['INDEX', '3', 'SORT', 'BY', 'Petal'

In [13]:
print(dg_formulas_sym)

[['(', ')'], ['(', ')'], ['(', ')'], ['(*)'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['>=', '.'], ['.', '.'], ['(', ')'], ['=', "'", "'"], ["'", "'"], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '],', ')'], ['([', '],', ')'], ['[', ']', '>=', '.'], ['[', ']', '<', '.'], ['[', ']', '!=', '.'], ['([', '],', '"', '")'], ['[', ']', '=', '.'], ['()', '<=', '([', '])'], ['()', '<=', '[', ']'], ['[', ']', '>=', '[', ']', '<='], ["('", "',", '[', '])'], ['([', '])', '=', '(', '())', '-'], ['[', ']', '>=', "('", "'", ',', '())'], ['([', '])', '[', ']', '[', ']'], ['(', '([', ']', ')', '=', "'", "'", '[', ']', ')'], ['[', ']', '>', '.', '[', ']', '>', '.'], ['([', '])', '<=', '.'], ['([', '])'], ['([', '])'], ['{', '[', ']', '=', '"', '"', ':', '([', '])', '}'], ['(', '[', ']', '=', '"', '"', ')'], ['(', '[', ']', '>', '.', '[', ']', ')'], ['{', '[', ']', '=', '"', '"', ':', '([', '])', '}'], ['{', '[',

In [14]:
dg_tok.drop(dg_tok.columns[1], axis=1)

,Tokenized Description
0,"[Sum, of, Sepal, Length]"
1,"[Average, Sepal, Width, across, all, entries]"
2,"[Find, the, median, Petal, Length]"
3,"[Count, the, number, of, records, in, the, dat..."
4,"[Count, distinct, species, in, the, dataset]"
5,"[Minimum, value, of, Sepal, Length]"
6,"[Maximum, Petal, Width, found, in, the, dataset]"
7,"[Group, records, by, Species]"
8,"[Sort, the, entries, by, Sepal, Length, in, as..."
9,"[List, Species, alphabetically]"


Adding columns to dg_tok of Formulas and Symbols in separated way

In [15]:
dg_tok["Tokenized Formula (Tableau)"]=dg_formulas_text

In [16]:
dg_tok["Tokenized Symbols (Tableau)"]=dg_formulas_sym

In [17]:
print(dg_tok.head(10))

                               Tokenized Description  \
0                           [Sum, of, Sepal, Length]   
1      [Average, Sepal, Width, across, all, entries]   
2                 [Find, the, median, Petal, Length]   
3  [Count, the, number, of, records, in, the, dat...   
4       [Count, distinct, species, in, the, dataset]   
5                [Minimum, value, of, Sepal, Length]   
6   [Maximum, Petal, Width, found, in, the, dataset]   
7                      [Group, records, by, Species]   
8  [Sort, the, entries, by, Sepal, Length, in, as...   
9                    [List, Species, alphabetically]   

       Tokenized Formula (Tableau) Tokenized Symbols (Tableau)  
0             [SUM, Sepal, Length]                      [(, )]  
1              [AVG, Sepal, Width]                      [(, )]  
2          [MEDIAN, Petal, Length]                      [(, )]  
3                          [COUNT]                       [(*)]  
4       [COUNT, DISTINCT, Species]                      [(

In [18]:
print(nlp.pipe(dg_tok[dg_tok.columns[0]]))

<generator object Language.pipe at 0x7b1888875690>


# Recognition of entities

In [19]:
def get_pos_(data,col_name):
  dd=[]
  gg=[]
  for tok in data[col_name]:

          for ent in nlp(str(list(str(tok).split()))):
              # checking text between punct
              dd.append({ent.text:ent.pos_})
              gg.append({ent.pos_:ent.text})
  return dd,gg

In [20]:
def get_patterns(data,col_name):
     dt,dp=get_pos_(data,col_name)

     pos_=[list(item.keys())[0] for item in dp]
     pos_=list(set(pos_))
     print(dt)
     print(col_name,":",pos_)
     patterns={}
     for key in pos_:
         patterns.update({key:list(set([d[key] for d in dp if  key==list(d.keys())[0]]))})
     return patterns

In [21]:
dg_ent={}

In [22]:
p_d=dg_ent['Patterns_Description']=get_patterns(dg_tok,dg_tok.columns[0])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Sum': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'of': 'ADP'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Average': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'across': 'ADP'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'all': 'PRO

In [23]:
for pat in p_d:
   print({pat:p_d[pat]})

{'ADV': ['exactly', 'alphabetically', 'only', 'Sort']}
{'INTJ': ['%']}
{'PART': ['not', "'", 'to']}
{'PRON': ['all', 'each', 'both', 'this', 'the', 'a', 'What', 'there']}
{'ADP': ['for', 'within', 'over', 'than', 'with', 'across', 'by', 'at', 'in', 'of', 'between', 'above', 'from']}
{'SCONJ': ['How', 'where', 'how']}
{'VERB': ['percentile', 'recorded', "'", 'sorted', 'found', 'show', 'Show', 'ascending', 'dataset', 'Date', 'assuming', 'Setosa', 'ends', 'start', 'descending', 'starts', 'Find', 'ica', 'setosa', 'Aggregate', 'Calculate', 'top', 'starting', 'contains', 'count', 'grouped', 'Organize']}
{'PUNCT': [',', ']', "'", '"', '(', '.', ')', '?']}
{'NOUN': ['value', 'Lengths', 'color', 'Determine', 'records', 'month', 'List', 'year', 'C', 'flowers', 'name', 'observation', 'Entries', 'Number', 'flower', 'points', 'types', 'versicolor', 'Species', 'field', 'data', 'entries', 'Sepal', 'Display', 'species', 'occurrences', 'number', 'Widths', 'order']}
{'AUX': ['is', "'s", 'are']}
{'CCONJ'

In [24]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(dg_tok,dg_tok.columns[1])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'SUM': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'AVG': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'MEDIAN': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Petal': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'":

In [25]:
for pat in p_t:
    print({pat:p_t[pat]})

{'ADV': ['alphabetically', 'ELSE', 'THEN', 'SORT']}
{'NUM': ['9', 'TOP', "'", '3', '5', '4']}
{'PART': ['to']}
{'SCONJ': ['IF']}
{'ADP': ['WITH', 'BY', 'BETWEEN', 'by', 'from']}
{'VERB': ['listed', 'STARTSWITH', 'ENDSWITH', "'", 'RANK', 'NULL', 'sorted', 'color', 'Date', 'setosa', 'SUM', 'DESC', 'DISTINCT', 'CONTAINS']}
{'NOUN': ['Species', 'year', 'ENDSWITH', 'ORDER', 'Sepal', 'TODAY', 'C', "'", 'FIXED', 'Date', 'SUM', 'YEAR', 'month', 'versicolor']}
{'PUNCT': [',', ']', "'", '"']}
{'AUX': ['RANK_PERCENTILE']}
{'CCONJ': ['AND']}
{'PROPN': ["'", 'COUNTD', 'AVG', 'MAX', 'Width', 'Date', 'Setosa', 'GROUP', 'Se', 'LEFT', 'ASC', 'END', 'STARTS', 'INDEX', 'Iri', 'Species', 'MIN', 'COUNT', 'Length', 'DATETRUNC', 'MEDIAN']}
{'DET': ['a']}
{'X': ['2', '1', "'", 'Length', '0', 'virginica', 'ica', '[']}
{'ADJ': ['Petal', 'highest', 'lowest']}


In [26]:
p_e=dg_ent['Patterns_Symbols(Tableau)']=get_patterns(dg_tok,dg_tok.columns[2])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PROPN'}, {'(': 'PUNCT'}, {'*': 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',

In [27]:
for pat in p_e:
    print({pat:p_e[pat]})

{'PRON': ['}']}
{'VERB': ["'", '=']}
{'PUNCT': [',', ']', '”', "'", '"', '{', '“', '(', ':', '.', ')', '*', '=', '!']}
{'NOUN': ['\\\'")\\']}
{'NUM': ["'"]}
{'X': ['>', ']', "'", '<', '=', '[']}
{'PROPN': ['-', '\\', '\\\'"\\', "'"]}


In [28]:
POS_=[]
for pat in p_d:
   POS_.append(pat)
for pat in p_t:
   POS_.append(pat)
for pat in p_e:
   POS_.append(pat)
POS_=list(set(POS_))
print(POS_, len(POS_))

['ADV', 'INTJ', 'PART', 'PRON', 'ADP', 'SCONJ', 'VERB', 'PUNCT', 'NOUN', 'AUX', 'CCONJ', 'NUM', 'DET', 'X', 'PROPN', 'ADJ'] 16


In [29]:
ENT_={}

for pat in POS_:
  if (p_d.get(pat)!=None and p_t.get(pat)!=None and p_e.get(pat) !=None):
    p_list=p_d[pat]+p_t[pat]+p_e[pat]
  elif (p_d.get(pat)!=None and p_t.get(pat)!=None ):
    p_list=p_d[pat] + p_t[pat]
  else:
    p_list=p_d[pat]

  ENT_.update({pat:[x for x in list(set(p_list))]})

In [30]:
for pat in POS_:
  print ({pat:ENT_[pat]})

{'ADV': ['THEN', 'SORT', 'Sort', 'only', 'ELSE', 'exactly', 'alphabetically']}
{'INTJ': ['%']}
{'PART': ['not', "'", 'to']}
{'PRON': ['all', 'each', 'both', 'this', 'the', 'a', 'What', 'there']}
{'ADP': ['for', 'over', 'than', 'WITH', 'of', 'BY', 'with', 'across', 'BETWEEN', 'by', 'at', 'in', 'within', 'between', 'above', 'from']}
{'SCONJ': ['How', 'where', 'IF', 'how']}
{'VERB': ['percentile', 'recorded', "'", 'RANK', 'sorted', 'color', 'found', 'DESC', 'show', 'Show', 'STARTSWITH', 'ascending', 'dataset', 'NULL', 'Date', 'assuming', 'SUM', 'Setosa', 'ends', 'CONTAINS', 'listed', 'start', 'ENDSWITH', 'descending', 'setosa', 'starts', 'Find', 'ica', 'DISTINCT', 'Aggregate', 'Calculate', 'top', 'starting', 'contains', 'count', '=', 'grouped', 'Organize']}
{'PUNCT': ['*', ',', ']', '”', "'", '"', '{', '“', '(', ':', '.', ')', '?', '=', '!']}
{'NOUN': ['value', 'ORDER', "'", 'Lengths', 'color', 'Determine', 'records', 'month', 'List', 'year', 'C', 'flowers', 'Date', 'SUM', 'name', 'ENDSWI

#Tokenize with Pre-trained sPacy models

In [31]:
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

# Description Patterns

In [32]:
dg_ent={}

In [33]:
p_d=dg_ent['Patterns_Description']=get_patterns(dg_tok,dg_tok.columns[0])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Sum': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'of': 'ADP'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Average': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'across': 'ADP'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'all': 'PRO

In [34]:
for pat in p_d:
   print({pat:p_d[pat]})

{'ADV': ['exactly', 'alphabetically', 'only', 'Sort']}
{'INTJ': ['%']}
{'PART': ['not', "'", 'to']}
{'PRON': ['all', 'each', 'both', 'this', 'the', 'a', 'What', 'there']}
{'ADP': ['for', 'within', 'over', 'than', 'with', 'across', 'by', 'at', 'in', 'of', 'between', 'above', 'from']}
{'SCONJ': ['How', 'where', 'how']}
{'VERB': ['percentile', 'recorded', "'", 'sorted', 'found', 'show', 'Show', 'ascending', 'dataset', 'Date', 'assuming', 'Setosa', 'ends', 'start', 'descending', 'starts', 'Find', 'ica', 'setosa', 'Aggregate', 'Calculate', 'top', 'starting', 'contains', 'count', 'grouped', 'Organize']}
{'PUNCT': [',', ']', "'", '"', '(', '.', ')', '?']}
{'NOUN': ['value', 'Lengths', 'color', 'Determine', 'records', 'month', 'List', 'year', 'C', 'flowers', 'name', 'observation', 'Entries', 'Number', 'flower', 'points', 'types', 'versicolor', 'Species', 'field', 'data', 'entries', 'Sepal', 'Display', 'species', 'occurrences', 'number', 'Widths', 'order']}
{'AUX': ['is', "'s", 'are']}
{'CCONJ'

# Formulas (Tableau) Patterns

In [35]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(dg_tok,dg_tok.columns[1])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'SUM': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'AVG': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'MEDIAN': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Petal': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'":

In [36]:
for pat in p_t:
   print({pat:p_t[pat]})

{'ADV': ['alphabetically', 'ELSE', 'THEN', 'SORT']}
{'NUM': ['9', 'TOP', "'", '3', '5', '4']}
{'PART': ['to']}
{'SCONJ': ['IF']}
{'ADP': ['WITH', 'BY', 'BETWEEN', 'by', 'from']}
{'VERB': ['listed', 'STARTSWITH', 'ENDSWITH', "'", 'RANK', 'NULL', 'sorted', 'color', 'Date', 'setosa', 'SUM', 'DESC', 'DISTINCT', 'CONTAINS']}
{'NOUN': ['Species', 'year', 'ENDSWITH', 'ORDER', 'Sepal', 'TODAY', 'C', "'", 'FIXED', 'Date', 'SUM', 'YEAR', 'month', 'versicolor']}
{'PUNCT': [',', ']', "'", '"']}
{'AUX': ['RANK_PERCENTILE']}
{'CCONJ': ['AND']}
{'PROPN': ["'", 'COUNTD', 'AVG', 'MAX', 'Width', 'Date', 'Setosa', 'GROUP', 'Se', 'LEFT', 'ASC', 'END', 'STARTS', 'INDEX', 'Iri', 'Species', 'MIN', 'COUNT', 'Length', 'DATETRUNC', 'MEDIAN']}
{'DET': ['a']}
{'X': ['2', '1', "'", 'Length', '0', 'virginica', 'ica', '[']}
{'ADJ': ['Petal', 'highest', 'lowest']}


#Tutorial of sPacy

In [37]:
text="Sum of Sepal Length SUM (Sepal Length)"
doc=nlp(text)
for token in doc.ents:
   print(token.text, token.label_,token.start_char,token.end_char, token.start, token.ent_id_, token.label, token.vector_norm)
for token in doc:
   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
           token.shape_, token.is_alpha, token.is_stop,token.ent_iob_, token.ent_type_)

Sepal Length PERSON 25 37 6  380 7.3880568
Sum Sum PROPN NNP nmod Xxx True False O 
of of ADP IN prep xx True True O 
Sepal Sepal PROPN NNP compound Xxxxx True False O 
Length Length PROPN NNP pobj Xxxxx True False O 
SUM SUM PROPN NNP ROOT XXX True False O 
( ( PUNCT -LRB- punct ( False False O 
Sepal Sepal PROPN NNP compound Xxxxx True False B PERSON
Length Length PROPN NNP appos Xxxxx True False I PERSON
) ) PUNCT -RRB- punct ) False False O 


In [38]:
# Create a span for the new entity
fb_ent = span.Span(doc, 0, 1, label="ORG")
orig_ents = list(doc.ents)

In [39]:
# Option 2: Assign a complete list of ents to doc.ents
doc.ents = orig_ents + [fb_ent]

In [40]:
# Add special case rule
from spacy.symbols import ORTH
special_case = [{ORTH: "gim"}, {ORTH: "me"}]
nlp.tokenizer.add_special_case("gimme", special_case)


In [41]:
# Tokenization and POS tagging result
tokens = [("Sum", "VERB"), ("of", "ADP"), ("Sepal", "NOUN"), ("Length", "NOUN")]
# Example mappings based on POS tagging results
operation_mapping = {"VERB": {"Sum": "SUM", "Average": "AVG", "Count": "COUNT"}}
field_mapping = {"NOUN": ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width","Species"]}


# TRAINING DATA FORMAT
TRAIN_DATA = []

for example in your_extracted_data:  # see example above
    entities = []

    for entity in example['data']:  # iterate over the entities
    
        text, label, start, end = entity  # ('want', '@command', 2, 6)
    
        label = label.split('@')[1].upper()  # not necessary, but nicer
    
        end = end - 1  # correct the end character index
    
        entities.append((start, end, label))
    
    # add training example of (text, annotations) tuple
    
    TRAIN_DATA.append((example['sentence'], {'entities': entities}))

This should give you training data that looks like this:

[
    ('I want apples', {'entities': [(2, 5, 'COMMAND'), (7, 12, 'FRUIT')]})
]


# DEFINE ENTITY RULER BEFORE NER

One potential difficulty in your example is that it's not very close to natural language. The pre-trained English models were trained on ~2m words of general web and news text, so they're not always going to perform perfect out-of-the-box on text with a very different structure.

While you could update the model with more examples of QUANTITY in your specific texts, I think that a rule-based approach might actually be a better and more efficient solution here.

The example in this blog post is actually very close to what you're trying to do:

import spacy

from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")

weights_pattern = [
    {"LIKE_NUM": True},

    {"LOWER": {"IN": ["g", "kg", "grams", "kilograms", "lb", "lbs", "pounds"]}}
]
patterns = [{"label": "QUANTITY", "pattern": weights_pattern}]

ruler = EntityRuler(nlp, patterns=patterns)

nlp.add_pipe(ruler, before="ner")

doc = nlp("U.S. average was 2 lbs.")

print([(ent.text, ent.label_) for ent in doc.ents])

 [('U.S.', 'GPE'), ('2 lbs', 'QUANTITY')]

The statistical named entity recognizer respects pre-defined entities and wil "predict around" them. So if you're adding the EntityRuler before it in the pipeline, your custom QUANTITY entities will be assigned first and will be taken into account when the entity recognizer predicts labels for the remaining tokens.

Note that this example is using the latest version of spaCy, v2.1.x. You might also want to add more patterns to cover different constructions. For more details and inspiration, check out the documentation on the EntityRuler, combining models and rules and the token match pattern syntax.

from spacy.lang.en import English

from spacy.matcher import Matcher

from spacy.tokens import Span

nlp = English()

matcher = Matcher(nlp.vocab)

def add_event_ent(matcher, doc, i, matches):

    # Get the current match and create tuple of entity label, start and end.

    # Append entity to the doc's entity. (Don't overwrite doc.ents!)

    match_id, start, end = matches[i]

    entity = Span(doc, start, end, label="EVENT")

    doc.ents += (entity,)

    print(entity.text)

pattern = [{"ORTH": "Google"}, {"ORTH": "I"}, {"ORTH": "/"}, {"ORTH": "O"}]

matcher.add("GoogleIO", [pattern], on_match=add_event_ent)

doc = nlp("This is a text about Google I/O")

matches = matcher(doc)

# RULE ENTITY DIRECTLY


# USE OF MACHER

nlp = spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)

Add match ID "HelloWorld" with no callback and one pattern

pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]

matcher.add("HelloWorld", [pattern])

doc = nlp("Hello, world! Hello world!")

matches = matcher(doc)

for match_id, start, end in matches:

    string_id = nlp.vocab.strings[match_id]  # Get string representation

    span = doc[start:end]  # The matched span

    print(match_id, string_id, start, end, span.text)


# VALIDATE ENTITY RULER
ruler = nlp.add_pipe("entity_ruler", config={"validate": True})


# ADD FUNCTION STRATEGY TO ADD A LOT OF NUMBER OF PATTERNS

ruler = nlp.add_pipe("entity_ruler")

patterns = [{"label": "TEST", "pattern": str(i)} for i in range(100000)]

with nlp.select_pipes(enable="tagger"):

    ruler.add_patterns(patterns)

#SPAN RULER
import spacy

nlp = spacy.blank("en")
ruler = nlp.add_pipe("span_ruler")
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]
ruler.add_patterns(patterns)

doc = nlp("Apple is opening its first big office in San Francisco.")
print([(span.text, span.label_) for span in doc.spans["ruler"]])


# PRE_TRAINED MODELS FOR SPACY

# load without NER

nlp = spacy.load("en_core_web_sm", exclude=["ner"])

# source NER from the same pipeline package as the last component

nlp.add_pipe("ner", source=spacy.load("en_core_web_sm"))

# insert the entity ruler

nlp.add_pipe("entity_ruler", before="ner")


# USAGE OF PRE-TRAINED MODELS
import spacy

nlp = spacy.load("en_core_web_sm")

import en_core_web_sm

nlp = en_core_web_sm.load()

doc = nlp("This is a sentence.")

print([(w.text, w.pos_) for w in doc])

# load without NER

nlp = spacy.load("en_core_web_sm", exclude=["ner"])

# source NER from the same pipeline package as the last component

nlp.add_pipe("ner", source=spacy.load("en_core_web_sm"))

# insert the entity ruler

nlp.add_pipe("entity_ruler", before="ner")